# Git Repository Bringup

In [4]:
!git version

git version 2.34.1


In [5]:
!git clone https://github.com/ramwtz/data-science-project.git

Cloning into 'data-science-project'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 132 (delta 55), reused 74 (delta 27), pack-reused 0
Receiving objects: 100% (132/132), 17.47 MiB | 16.72 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [6]:
!ls

data-science-project  sample_data


In [7]:
%cd data-science-project/

/content/data-science-project


In [8]:
!pwd

/content/data-science-project


In [9]:
!git switch dev/ram

Branch 'dev/ram' set up to track remote branch 'dev/ram' from 'origin'.
Switched to a new branch 'dev/ram'


In [10]:
!git pull

Already up to date.


In [11]:
%cd ..

/content


# Basic Bringup for Remote Environment

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# Necessary packages for this nb
!pip install --quiet --upgrade -r '/content/data-science-project/dependencies.txt'
!pip install pocketsphinx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 62.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━

# Initialize Env

In [14]:
# Imports
import speech_recognition as sr
from vosk import Model, KaldiRecognizer
import os
import librosa as lbrs
from tqdm import tqdm
import pandas as pd
import json
from difflib import SequenceMatcher
import timeit

# Configs
%load_ext autotime
tqdm.pandas()

time: 2.09 ms (started: 2023-09-29 18:55:32 +00:00)


In [15]:
gen_data_dir = os.path.join('drive', 'MyDrive', 'project', 'data')
calls_dir = os.path.join(gen_data_dir, 'calls')
new_data_dir = os.path.join(gen_data_dir, 'new')
models_path = os.path.join('drive', 'MyDrive', 'model')
metadata_path = os.path.join(calls_dir, '911_first6sec', '911_metadata_6sec.csv')

time: 861 µs (started: 2023-09-29 18:55:35 +00:00)


In [16]:
def get_path_from_filename(audio_file_name):
    type_dir, file_name = audio_file_name.split('/')
    return os.path.join(calls_dir, type_dir, file_name)

time: 550 µs (started: 2023-09-29 18:55:36 +00:00)


In [17]:
def create_audio_series(audio_file_name):
    audio_file_path = get_path_from_filename(audio_file_name)
    data, sample_rate = lbrs.load(audio_file_path)
    return data, sample_rate

time: 594 µs (started: 2023-09-29 18:55:37 +00:00)


# Fetching the Transcription
We're going to you the SpeechRecognition library to transform the audio to test to use as features for our model training

## Defs

In [18]:
r = sr.Recognizer()

time: 481 µs (started: 2023-09-29 18:55:58 +00:00)


In [19]:
def get_audio(audio_file_path, filter_noise=False):
  with sr.AudioFile(audio_file_path) as src:
    if filter_noise:
      r.adjust_for_ambient_noise(src, duration=0.5)
    return r.record(src)

time: 633 µs (started: 2023-09-29 18:55:59 +00:00)


In [20]:
def get_path_from_filename(audio_file_name):
    type_dir, file_name = audio_file_name.split('/')
    return os.path.join(calls_dir, type_dir, file_name)

time: 428 µs (started: 2023-09-29 18:55:59 +00:00)


## Raw Audio
We will first try to decipher the audio as is using four different speach to text engines: Google, Sphinx, Vosk & Whisper (engines that have public APIs).

### Aggregate Transcriptions

In [21]:
def recognize_vosk(audio, model=None):
  if model is None:
    model_path = os.path.join(models_path, 'vosk', 'eng_small')
  else:
    model_path = os.path.join(models_path, 'vosk', model)

  vosk_audio = audio.get_raw_data(convert_rate=16000, convert_width=2)

  #
  model = Model(model_path)
  recognizer = KaldiRecognizer(model, 16000)
  recognizer.AcceptWaveform(vosk_audio)
  return recognizer.FinalResult()

time: 808 µs (started: 2023-09-29 18:56:01 +00:00)


In [22]:
def transcribe_file(filename, filter_noise=False):
  path = get_path_from_filename(filename)
  audio = get_audio(path, filter_noise=filter_noise)

  try:
    g = r.recognize_google(audio)
  except:
    g = "UNABLE TO RECOGNIZE"

  s = r.recognize_sphinx(audio)

  w = r.recognize_whisper(audio)

  vt = recognize_vosk(audio)
  v = json.loads(vt)['text']

  return g, s, w, v

time: 807 µs (started: 2023-09-29 18:56:03 +00:00)


In [10]:
metadata_df = pd.read_csv(metadata_path)

time: 31.2 ms (started: 2023-09-29 18:05:09 +00:00)


In [11]:
manual_transcriptions = {
    '911_first6sec/call_8_0.wav': "um hello hello is everything okay",
    '911_first6sec/call_9_0.wav': 'yes um i need a police officer over here at whats going on',
    '911_first6sec/call_10_0.wav': "ma'am my pickup was stolen I had to go find it uhh",
    '911_first6sec/call_11_0.wav': "no ma'am i don't i don't have an emergency two police officers just at my house just now",
    '911_first6sec/call_15_0.wav': "I'm Dylon Peterson and i just killed two people",
    '911_first6sec/call_17_0.wav': "yeah we got a fire down here at on route seven right here at abington",
    '911_first6sec/call_20_0.wav': "are you out of the house yet no alright get out of the house we can't theres a",
    '911_first6sec/call_21_0.wav': "yes is this a police station yes it is may i help you yeah okay i put this is ridiculous",
    '911_first6sec/call_28_0.wav': "can you please send rescue to i think i'm",
    '911_first6sec/call_31_0.wav': "oh my god somebody is shooting at the sheriff's department at the sheriff department yes building?"
}

manual_transcriptions = pd.Series(manual_transcriptions)
manual_transcriptions = manual_transcriptions.rename('manual_transcriptions')
manual_transcriptions = pd.DataFrame(manual_transcriptions).reset_index()
manual_transcriptions = manual_transcriptions.rename({'index': 'filename'}, axis=1)

time: 7.54 ms (started: 2023-09-29 18:05:11 +00:00)


In [12]:
filenames = pd.Series(manual_transcriptions['filename'])

time: 739 µs (started: 2023-09-29 18:05:12 +00:00)


In [46]:
indices = ['google', 'sphinx', 'whisper', 'vosk']

transcriptions_series = filenames.progress_apply(transcribe_file)
column_names = {
    k: v for k, v in enumerate(indices)
  }
transcriptions_list = transcriptions_series.values.tolist()
transcriptions = pd.DataFrame(transcriptions_list).rename(column_names, axis=1)

100%|██████████| 10/10 [01:59<00:00, 11.93s/it]

time: 1min 59s (started: 2023-09-29 14:09:34 +00:00)


In [47]:
assessment = pd.DataFrame({"filename": filenames})
assessment = pd.concat([assessment, transcriptions], axis=1)

time: 1.98 ms (started: 2023-09-29 14:11:36 +00:00)


In [48]:
assessment = assessment.merge(manual_transcriptions, left_on='filename', right_on='filename')

time: 3.48 ms (started: 2023-09-29 14:11:38 +00:00)


In [49]:
assessment

,filename,google,sphinx,whisper,vosk,manual_transcriptions
0,911_first6sec/call_8_0.wav,hello hello is everything okay,and while the al and then again okay go ahead,Hello. Is everything okay? On.,ah now i now it's everything okay,um hello hello is everything okay
1,911_first6sec/call_9_0.wav,UNABLE TO RECOGNIZE,he had an idea of a pound and a coil of oil an...,"Yes, I need a police officer over here. What'...",that i'm anita police officer over here the co...,yes um i need a police officer over here at wh...
2,911_first6sec/call_10_0.wav,damn my pickup was born I had to go find it,no wonder the political had invited laugh,"Ma'am, my cat is a person. I had it offended.",now my pick up on had to go find that ah,ma'am my pickup was stolen I had to go find it...
3,911_first6sec/call_11_0.wav,UNABLE TO RECOGNIZE,i'd really a man am i to have an latent he cou...,I'm gonna go to the bathroom and close the le...,remember my mom i got my house now out,no ma'am i don't i don't have an emergency two...
4,911_first6sec/call_15_0.wav,I'm done Peterson and I just killed two people,glendon beard and that is killed two people that,I'm Joan Pearson and that has killed two people.,i'm i'm down there and that it killed few people,I'm Dylon Peterson and i just killed two people
5,911_first6sec/call_17_0.wav,yeah we got a fire down here it's on Route 7 r...,the little hard and long little doubt and let ...,"We've got a far down here, it's around 7, rig...",there we go too far down here at our or out th...,yeah we got a fire down here at on route seven...
6,911_first6sec/call_20_0.wav,are you out of the house yet no or I get out o...,we have our dead and now or i didn't have reac...,Are you out of the house yet? No. Or I get ou...,i get out of our now i get out of as we can't,are you out of the house yet no alright get ou...
7,911_first6sec/call_21_0.wav,UNABLE TO RECOGNIZE,whether you're completely hemmed and looking f...,"Yes, it is. Please return. Yes, it is. May he...",i can't help you okay i put is ridiculous,yes is this a police station yes it is may i h...
8,911_first6sec/call_28_0.wav,can you please send rescue to I think I'm,and i knew her out and adam had,Can you please send rescue to... I think I'm ...,you prefer back to to i think i'm having,can you please send rescue to i think i'm
9,911_first6sec/call_31_0.wav,oh my God somebody shooting at the sheriff's d...,so why when you hear more than that but i'd ha...,"Oh my God, so many students at this year's de...",oh look at how many shooting here for partner ...,oh my god somebody is shooting at the sheriff'...


time: 14.9 ms (started: 2023-09-29 14:11:39 +00:00)


### Assess Transcriptions

In [13]:
def similar(row, indices: list, compare_to='manual_transcriptions'):
    t = [
        SequenceMatcher(a=row[col], b=row[compare_to]).ratio()
        for col in indices
    ]
    return tuple(t)

time: 723 µs (started: 2023-09-29 18:05:17 +00:00)


In [52]:
scores_series = assessment.progress_apply(similar, axis=1, indices=indices)
scores_list = scores_series.values.tolist()
scores = pd.DataFrame(scores_list).rename(column_names, axis=1)

100%|██████████| 10/10 [00:00<00:00, 710.95it/s]

time: 23.3 ms (started: 2023-09-29 14:12:00 +00:00)


In [56]:
print(scores.mean().sort_values(ascending=False))

whisper    0.682378
google     0.639828
vosk       0.593321
sphinx     0.334570
dtype: float64
time: 4.41 ms (started: 2023-09-29 14:12:59 +00:00)


We can see that whisper has the best score, followed by google and vosk, while sphinx lags behind.

## Clean Audio

### Aggregate Transcriptions

In [57]:
transcriptions_series = filenames.progress_apply(transcribe_file, filter_noise=True)
transcriptions_list = transcriptions_series.values.tolist()
transcriptions = pd.DataFrame(transcriptions_list).rename(column_names, axis=1)

100%|██████████| 10/10 [01:51<00:00, 11.16s/it]

time: 1min 51s (started: 2023-09-29 14:13:21 +00:00)


In [58]:
assessment_clean = pd.DataFrame({"filename": filenames})
assessment_clean = pd.concat([assessment_clean, transcriptions], axis=1)

time: 1.73 ms (started: 2023-09-29 14:15:23 +00:00)


In [59]:
assessment_clean = assessment_clean.merge(manual_transcriptions, left_on='filename', right_on='filename')

time: 4.2 ms (started: 2023-09-29 14:15:25 +00:00)


In [60]:
assessment_clean

,filename,google,sphinx,whisper,vosk,manual_transcriptions
0,911_first6sec/call_8_0.wav,put on hello hello is everything okay,and while canal in the making of the coup,Hello. Hello. Is everything okay? I'm.,now i'm now it's everything okay,um hello hello is everything okay
1,911_first6sec/call_9_0.wav,what's going on,and i had had a war what owen adam had,I need a police officer over here at 7. What'...,anita a police officer of the here the corner ...,yes um i need a police officer over here at wh...
2,911_first6sec/call_10_0.wav,damn my pickup was born I had to go find it,my own mind at football has invited up,"Ma'am, my kekep was born. I had to go find it.",well my pick up on had to go find that ah,ma'am my pickup was stolen I had to go find it...
3,911_first6sec/call_11_0.wav,UNABLE TO RECOGNIZE,i'd really a man am i to have an latent he cou...,I'm gonna go to the bathroom and close the le...,remember my mom i got my house now,no ma'am i don't i don't have an emergency two...
4,911_first6sec/call_15_0.wav,I'm downstairs and I just killed two people,one delegate in the uphill to people that,I'm Don Pearson and I just killed two people.,i'm down here and that it killed people,I'm Dylon Peterson and i just killed two people
5,911_first6sec/call_17_0.wav,on Route 7 right here in Abington,god that the loaded on little doubt and let it,"We've got a far down here, it's on Route 7, r...",the far down here or out thousand right here t...,yeah we got a fire down here at on route seven...
6,911_first6sec/call_20_0.wav,get out of the house yet no I get out of the h...,that i'd have now or i didn't have you read th...,You're out of the house yet? No. Or I get out...,the out of as you know i get out of as we can't,are you out of the house yet no alright get ou...
7,911_first6sec/call_21_0.wav,UNABLE TO RECOGNIZE,i have heard of klingon camp but little more k...,"Yes, it is a police station. Yes, it is. May ...",i can't say help you okay i put the ridiculous,yes is this a police station yes it is may i h...
8,911_first6sec/call_28_0.wav,please send rescue to I think I'm,and i do her out and adam had,Please send a rescue to I think I'm having a,that do too i think i'm heading,can you please send rescue to i think i'm
9,911_first6sec/call_31_0.wav,oh my God somebody shooting at the sheriff's d...,i'm alive than he wanted identify have been he...,"Oh my God, some of these children that they s...",oh my god how many hidden a partner at the sha...,oh my god somebody is shooting at the sheriff'...


time: 20.9 ms (started: 2023-09-29 14:15:27 +00:00)


### Assess Transcriptions

In [62]:
scores_clean_series = assessment_clean.progress_apply(similar, axis=1, indices=indices)
scores_clean_list = scores_clean_series.values.tolist()
scores_clean = pd.DataFrame(scores_clean_list).rename(column_names, axis=1)

100%|██████████| 10/10 [00:00<00:00, 582.35it/s]

time: 24 ms (started: 2023-09-29 14:15:48 +00:00)


In [63]:
print(scores_clean.mean().sort_values(ascending=False))

whisper    0.741748
google     0.617476
vosk       0.594663
sphinx     0.319726
dtype: float64
time: 3.63 ms (started: 2023-09-29 14:15:52 +00:00)


We essentially got the same ranking, with google and sphinx actually worsening as a result of the cleaning, vosk staying roughly the same and whisper improving substantially.

Because Google is unreliable in producing a transcription, we will procced with whisper and vosk and check some of the different models available.

## Checking Vosk & Whisper Models

### VOSK

#### Aggregate Transcriptions

In [14]:
def transcribe_vosk(filename, filter_noise=False, models=[]):
  path = get_path_from_filename(filename)
  audio = get_audio(path, filter_noise=filter_noise)

  t = []
  for model in models:
    vt = recognize_vosk(audio, model)
    t.append(json.loads(vt)['text'])

  return tuple(t)

time: 642 µs (started: 2023-09-29 18:05:21 +00:00)


In [16]:
vosk_models = ['eng_large', 'eng_small', 'vosk-model-en-us-0.22-lgraph', 'vosk-model-en-us-0.42-gigaspeech']
vosk_series = filenames.progress_apply(transcribe_vosk, filter_noise=True, models=vosk_models)
vosk_column_names = {
    k: v for k, v in enumerate(vosk_models)
  }
vosk_list = vosk_series.values.tolist()
vosk_df = pd.DataFrame(vosk_list).rename(vosk_column_names, axis=1)

100%|██████████| 10/10 [16:56<00:00, 101.63s/it]

time: 16min 56s (started: 2023-09-29 16:26:32 +00:00)


In [17]:
assessment_vosk = pd.DataFrame({"filename": filenames})
assessment_vosk = pd.concat([assessment_vosk, vosk_df], axis=1)

time: 2.17 ms (started: 2023-09-29 16:43:44 +00:00)


In [18]:
assessment_vosk = assessment_vosk.merge(manual_transcriptions, left_on='filename', right_on='filename')

time: 74 ms (started: 2023-09-29 16:43:46 +00:00)


In [19]:
assessment_vosk

,filename,eng_large,eng_small,vosk-model-en-us-0.22-lgraph,vosk-model-en-us-0.42-gigaspeech,manual_transcriptions
0,911_first6sec/call_8_0.wav,i know i know is everything ok,now i'm now it's everything okay,i hello i know is everything okay,um hello hello is everything okay huh,um hello hello is everything okay
1,911_first6sec/call_9_0.wav,i need a police officer over here except what'...,anita a police officer of the hear the corner ...,i need a police officer over here it's of coin...,i need a police officer over here at some reco...,yes um i need a police officer over here at wh...
2,911_first6sec/call_10_0.wav,well my pickup was gone i had to go find it,well my pick up on a go find that,well my a post gone i to go find that,well my pickup was gone i had to go find it ah,ma'am my pickup was stolen I had to go find it...
3,911_first6sec/call_11_0.wav,i remember the man again i who am i to please ...,remember my mom i got my house personality,i remember a snowman again i who am i going to...,unfortunately no ma'am i don't want emergency ...,no ma'am i don't i don't have an emergency two...
4,911_first6sec/call_15_0.wav,i am down there and i just killed two people,i'm down here and that it killed people,i i'm down there in and that as killed two people,um i'm down here in ny this killed two people,I'm Dylon Peterson and i just killed two people
5,911_first6sec/call_17_0.wav,as far down here at all on route southern righ...,the far down here at or out thousand right her...,at the far down here at all on route southern ...,so far down here it's on route seven right her...,yeah we got a fire down here at on route seven...
6,911_first6sec/call_20_0.wav,out of the house yet no or i get out of the ho...,the out of as you know i get out of as we can't,it out of the house yet no but i get out of th...,you're out of the house yes no i get out of th...,are you out of the house yet no alright get ou...
7,911_first6sec/call_21_0.wav,i think i think may help you here okay i put r...,i can't say help you okay i put the ridiculous,in current events may help you here okay i put...,guess it is simply thinking yes it is may help...,yes is this a police station yes it is may i h...
8,911_first6sec/call_28_0.wav,we sent back due to epic i'm having,that do too i think i'm heading,we been back due to i think i'm having,please send rescue to i think i'm heading,can you please send rescue to i think i'm
9,911_first6sec/call_31_0.wav,oh my god somebody shooting at the sheriff's d...,oh my god how many hidden a partner at the sha...,oh my god somebody shooting at the sheriff's d...,oh my god somebody shooting the sheriff's depa...,oh my god somebody is shooting at the sheriff'...


time: 22.7 ms (started: 2023-09-29 16:43:47 +00:00)


#### Assess Transcriptions

In [27]:
scores_vosk_series = assessment_vosk.progress_apply(similar, axis=1, indices=vosk_models)
scores_vosk_list = scores_vosk_series.values.tolist()
scores_vosk = pd.DataFrame(scores_vosk_list).rename(vosk_column_names, axis=1)

100%|██████████| 10/10 [00:00<00:00, 539.26it/s]

time: 25.8 ms (started: 2023-09-29 16:45:08 +00:00)


In [28]:
print(scores_vosk.mean().sort_values(ascending=False))

vosk-model-en-us-0.42-gigaspeech    0.806011
eng_large                           0.728483
vosk-model-en-us-0.22-lgraph        0.706237
eng_small                           0.582618
dtype: float64
time: 2.31 ms (started: 2023-09-29 16:45:11 +00:00)


We've got some pretty good performances here from the heavy models, with the gigaspeech performing very well.

### Whisper

#### Aggregate Transcriptions

In [23]:
def transcribe_whisper(filename, filter_noise=False, models=[]):
  path = get_path_from_filename(filename)
  audio = get_audio(path, filter_noise=filter_noise)

  t = []
  for model in models:
      t.append(r.recognize_whisper(audio, model))

  return tuple(t)

time: 653 µs (started: 2023-09-29 18:56:11 +00:00)


In [21]:
whisper_models = [
    'base', 'medium', 'large'
    ]
whisper_series = filenames.progress_apply(transcribe_whisper, filter_noise=True, models=whisper_models)
whisper_column_names = {
    k: v for k, v in enumerate(whisper_models)
  }
whisper_list = whisper_series.values.tolist()
whisper_df = pd.DataFrame(whisper_list).rename(whisper_column_names, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]
  0%|                                              | 0.00/2.87G [00:00<?, ?iB/s]
  0%|                                     | 5.48M/2.87G [00:00<00:53, 57.4MiB/s]
  1%|▎                                     | 22.9M/2.87G [00:00<00:23, 131MiB/s]
  1%|▍                                     | 36.6M/2.87G [00:00<00:22, 137MiB/s]
  2%|▋                                     | 55.6M/2.87G [00:00<00:18, 161MiB/s]
  2%|▉                                     | 71.8M/2.87G [00:00<00:18, 164MiB/s]
  3%|█▏                                    | 91.6M/2.87G [00:00<00:16, 179MiB/s]
  4%|█▍                                     | 109M/2.87G [00:00<00:17, 173MiB/s]
  4%|█▋                                     | 125M/2.87G [00:00<00:17, 169MiB/s]
  5%|█▉                                     | 144M/2.87G [00:00<00:16, 178MiB/s]
  6%|██▏                                    | 162M/2.87G [00:01<00:16, 182MiB/s]
  6%|██▍                                    | 180M/2.87G [00:01<00:16,

time: 3min 18s (started: 2023-09-29 18:24:52 +00:00)


In [22]:
whisper_assessment = pd.DataFrame({"filename": filenames})
whisper_assessment = pd.concat([whisper_assessment, whisper_df], axis=1)

time: 1.9 ms (started: 2023-09-29 18:28:11 +00:00)


In [23]:
whisper_assessment = whisper_assessment.merge(manual_transcriptions, left_on='filename', right_on='filename')

time: 71.6 ms (started: 2023-09-29 18:28:11 +00:00)


In [24]:
whisper_assessment

,filename,base,medium,large,manual_transcriptions
0,911_first6sec/call_8_0.wav,Hello. Hello. Is everything okay? I'm.,Hello. Hello. Is everything okay? Um.,"Um, hello. Hello. Is everything okay? Um... ...",um hello hello is everything okay
1,911_first6sec/call_9_0.wav,I need a police officer over here at 7. What'...,I need a police officer over here at 7th. Wha...,I need a police officer over here at 7th and ...,yes um i need a police officer over here at wh...
2,911_first6sec/call_10_0.wav,"Ma'am, my kekep was born. I had to go find it.","Ma'am, my pickup was stolen. I had to go find...","Ma'am, my pickup was stolen. I had to go find...",ma'am my pickup was stolen I had to go find it...
3,911_first6sec/call_11_0.wav,I'm gonna go to the bathroom and close the le...,No ma'am I don't. I don't have an emergency. ...,No ma'am I don't. I don't have an emergency t...,no ma'am i don't i don't have an emergency two...
4,911_first6sec/call_15_0.wav,I'm Don Pearson and I just killed two people.,I'm Dylan Beards and that has killed two people.,I'm Dylan Pierce and I just killed two people.,I'm Dylon Peterson and i just killed two people
5,911_first6sec/call_17_0.wav,"We've got a far down here, it's on Route 7, r...","We've got a fire down here on Route 7, right ...","We've got a fire down here on Route 7, right ...",yeah we got a fire down here at on route seven...
6,911_first6sec/call_20_0.wav,You're out of the house yet? No. Or I get out...,"Are you out of the house yet? No. All right, ...",you out of the house yet?,are you out of the house yet no alright get ou...
7,911_first6sec/call_21_0.wav,"Yes, it is a police station. Yes, it is. May ...","Yes, is this the police station? Yes it is, m...","Yes, is this the police station? Yes, it is. ...",yes is this a police station yes it is may i h...
8,911_first6sec/call_28_0.wav,Please send a rescue to I think I'm having a,police and rescue to. I think I'm having.,Please send rescue to... I think I'm having...,can you please send rescue to i think i'm
9,911_first6sec/call_31_0.wav,"Oh my God, some of these children that they s...","Oh my god, so many students at the sheriff's ...","Oh my god, somebody's shooting at the Sheriff...",oh my god somebody is shooting at the sheriff'...


time: 13 ms (started: 2023-09-29 18:28:11 +00:00)


### Assess Transcriptions

In [36]:
whisper_scores_series = whisper_assessment.progress_apply(similar, axis=1, indices=whisper_models)
whisper_scores_list = whisper_scores_series.values.tolist()
whisper_scores = pd.DataFrame(whisper_scores_list).rename(whisper_column_names, axis=1)

100%|██████████| 10/10 [00:00<00:00, 443.49it/s]

time: 31.2 ms (started: 2023-09-29 16:52:10 +00:00)


In [37]:
print(whisper_scores.mean().sort_values(ascending=False))

medium    0.808133
large     0.755207
base      0.741748
dtype: float64
time: 8.7 ms (started: 2023-09-29 16:52:13 +00:00)


Interestingly the medium model performed better than the large model, with performance roughlt equivalent to the vosk gigaspeech model, lets comoare their time benhcmarks next.

### Vosk Gigaspeech vs Whisper Medium - Time Benchmark

In [17]:
def time_transcribe(filename, transcriber, models):
  timer = timeit.Timer(
    lambda: transcriber(filename, filter_noise=True, models=models))
  return timer.timeit(1)

time: 610 µs (started: 2023-09-29 18:06:32 +00:00)


In [23]:
whisper_models = ['medium']
whisper_time = filenames.progress_apply(time_transcribe, transcriber=transcribe_whisper, models=whisper_models)

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]

time: 51.3 s (started: 2023-09-29 18:01:20 +00:00)


In [24]:
transcribe_time = pd.DataFrame(filenames)
transcribe_time['whisper_time'] = whisper_time
transcribe_time

,filename,whisper_time
0,911_first6sec/call_8_0.wav,38.522525
1,911_first6sec/call_9_0.wav,1.369517
2,911_first6sec/call_10_0.wav,1.215789
3,911_first6sec/call_11_0.wav,1.404257
4,911_first6sec/call_15_0.wav,0.955023
5,911_first6sec/call_17_0.wav,1.127166
6,911_first6sec/call_20_0.wav,1.734471
7,911_first6sec/call_21_0.wav,1.755536
8,911_first6sec/call_28_0.wav,0.982879
9,911_first6sec/call_31_0.wav,2.022964


time: 18.9 ms (started: 2023-09-29 18:02:13 +00:00)


We can see a longer time for the first transcription, most likely as a result of loading the model to memory, but after that consistently fast performance.

In [18]:
vosk_models = ['vosk-model-en-us-0.42-gigaspeech']
vosk_time = filenames.progress_apply(time_transcribe, transcriber=transcribe_vosk, models=vosk_models)

100%|██████████| 10/10 [07:58<00:00, 47.83s/it]

time: 7min 58s (started: 2023-09-29 18:06:33 +00:00)


In [19]:
transcribe_time = pd.DataFrame(filenames)
transcribe_time['vosk_time'] = vosk_time
transcribe_time

,filename,vosk_time
0,911_first6sec/call_8_0.wav,44.812025
1,911_first6sec/call_9_0.wav,44.612384
2,911_first6sec/call_10_0.wav,49.292902
3,911_first6sec/call_11_0.wav,47.072907
4,911_first6sec/call_15_0.wav,43.161186
5,911_first6sec/call_17_0.wav,47.722744
6,911_first6sec/call_20_0.wav,52.146974
7,911_first6sec/call_21_0.wav,47.600496
8,911_first6sec/call_28_0.wav,49.787880
9,911_first6sec/call_31_0.wav,52.042737


time: 30.6 ms (started: 2023-09-29 18:15:19 +00:00)


This model is much slower, and with having almost equal accuracy to whisper, we will choose the whisper-medium model as our transcription model.

## Transcribing the Dataset

In [24]:
metadata_df = pd.read_csv(metadata_path)

time: 5.11 s (started: 2023-09-29 18:56:13 +00:00)


In [27]:
filenames = pd.Series(metadata_df['filename'].unique())

time: 1.37 ms (started: 2023-09-29 18:56:45 +00:00)


In [ ]:
whisper_models = ['medium']
whisper_series = filenames.progress_apply(
    transcribe_whisper, filter_noise=True, models=whisper_models)
whisper_column_names = {
    k: v for k, v in enumerate(whisper_models)
  }
whisper_list = whisper_series.values.tolist()
whisper_df = pd.DataFrame(whisper_list).rename(whisper_column_names, axis=1)

In [29]:
whisper_df

,medium
0,My mom had a bad time.
1,Hello. Hello. Is everything okay? Um.
2,I need a police officer over here at 7th. Wha...
3,"Ma'am, my pickup was stolen. I had to go find..."
4,No ma'am I don't. I don't have an emergency. ...
...,...
702,She's not waking up. Okay. What's her address?
703,I just shot the Mormon bishop over here on Tu...
704,And this guy has a gun in my hat. I need the ...
705,There are like at least four or five people w...


time: 35.6 ms (started: 2023-09-29 19:36:12 +00:00)


In [37]:
filenames = filenames.rename('filename')
filenames_df = pd.DataFrame(filenames)
transcriptions_df = pd.concat([filenames, whisper_df], axis=1)

time: 4.44 ms (started: 2023-09-29 19:39:22 +00:00)


In [38]:
transcriptions_df.head(10)

,filename,medium
0,911_first6sec/call_2_0.wav,My mom had a bad time.
1,911_first6sec/call_8_0.wav,Hello. Hello. Is everything okay? Um.
2,911_first6sec/call_9_0.wav,I need a police officer over here at 7th. Wha...
3,911_first6sec/call_10_0.wav,"Ma'am, my pickup was stolen. I had to go find..."
4,911_first6sec/call_11_0.wav,No ma'am I don't. I don't have an emergency. ...
5,911_first6sec/call_12_0.wav,shirt
6,911_first6sec/call_13_0.wav,"We're winning, ma'am. Police! What's going on..."
7,911_first6sec/call_15_0.wav,I'm Dylan Beards and that has killed two people.
8,911_first6sec/call_16_0.wav,"ambulance, the police officer at one. OK, you..."
9,911_first6sec/call_17_0.wav,"We've got a fire down here on Route 7, right ..."


time: 31.1 ms (started: 2023-09-29 19:39:36 +00:00)


In [40]:
path_to_transcr = os.path.join(calls_dir, '911_first6sec', 'transcriptions')
transcriptions_df.to_csv(path_to_transcr, index=False)

time: 28.8 ms (started: 2023-09-29 19:42:45 +00:00)


# Sentiment Analysis - OLD

### UTILS

In [ ]:
from nltk.corpus import stopwords
import pandas as pd
import string
from nltk.tokenize import RegexpTokenizer

def tokenize(raw):
    tokenizer = RegexpTokenizer(r"\w+\'?\w*")
    return tokenizer.tokenize(raw)

def preprocess_words_fast(tokens):
    t = pd.Series(tokens).str.lower()
    t = t[~(t.isin(stopwords.words('english')) | t.isin(list(string.punctuation)))]
    return t.tolist()

In [ ]:
raw = " Ma'am, my pickup was stolen. I had to go find it. Ahh!"
tokens = tokenize(raw)
words = preprocess_words_fast(tokens)
print(words)

["ma'am", 'pickup', 'stolen', 'go', 'find', 'ahh']


### vader

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
# dirty
sid.polarity_scores(raw)

{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'compound': -0.5411}

In [ ]:
# clean
sid.polarity_scores(' '.join(words))

{'neg': 0.39, 'neu': 0.61, 'pos': 0.0, 'compound': -0.4939}

### textblob

In [ ]:
from textblob import TextBlob

In [ ]:
# dirty
testimonial = TextBlob(raw)
print(testimonial.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


In [ ]:
# clean
testimonial = TextBlob(' '.join(words))
print(testimonial.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


### Flair

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

In [ ]:
classifier = TextClassifier.load('en-sentiment')

In [ ]:
sentence = Sentence(raw)
classifier.predict(sentence)

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

Sentence above is:  ['Sentence[16]: " Ma'am, my pickup was stolen. I had to go find it. Ahh!"'/'NEGATIVE' (0.9804)]


In [ ]:
sentence = Sentence(' '.join(words))
classifier.predict(sentence)

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

Sentence above is:  ['Sentence[6]: "ma'am pickup stolen go find ahh"'/'NEGATIVE' (0.9505)]


### Transformers

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis")

In [ ]:
# dirty
classifier([raw])

[{'label': 'NEGATIVE', 'score': 0.9987456798553467}]

In [ ]:
# clean
classifier([''.join(words)])

[{'label': 'NEGATIVE', 'score': 0.9751687049865723}]

# lets use transformers pipeline to analyze sentiment

In [ ]:
from transformers import pipeline

def get_sentiment_transformers(text):
    classifier = pipeline("sentiment-analysis")
    sent = classifier([text])
    return sent[0]['label']

In [ ]:
sentiment_df = text_series.apply(get_sentiment_transformers)
sentiment_df

0     NEGATIVE
1     POSITIVE
2     NEGATIVE
3     NEGATIVE
4     NEGATIVE
        ...   
95    NEGATIVE
96    POSITIVE
97    NEGATIVE
98    NEGATIVE
99    NEGATIVE
Name: filename, Length: 100, dtype: object

In [ ]:
full_df = pd.DataFrame([df['filename'][:100], text_series, sentiment_df]).T
full_df.columns=['filename', 'text', 'sentiment']
full_df

,filename,text,sentiment
0,911_first6sec/call_2_0.wav,My mom had a bad time.,NEGATIVE
1,911_first6sec/call_8_0.wav,Hello. Hello. Is everything okay?,POSITIVE
2,911_first6sec/call_9_0.wav,I need a police officer over here at 7th. Wha...,NEGATIVE
3,911_first6sec/call_10_0.wav,"Ma'am, my pickup was stolen. I had to go find...",NEGATIVE
4,911_first6sec/call_11_0.wav,No ma'am I don't. I don't have an emergency. ...,NEGATIVE
...,...,...,...
95,911_first6sec/call_110_0.wav,"This is Gifford. Hello? Hello? Oh, number one...",NEGATIVE
96,911_first6sec/call_111_0.wav,"across, forty seven oh nine fourty隔s avenue f...",POSITIVE
97,911_first6sec/call_112_0.wav,"I have, um, I need somebody escorted out of m...",NEGATIVE
98,911_first6sec/call_114_0.wav,Hello? Hello? Hello? Hello 911? Hello?,NEGATIVE


In [ ]:
full_df[full_df['sentiment'] == 'NEGATIVE'].count()

filename     78
text         78
sentiment    78
dtype: int64

# lets use Vader pipeline to analyze sentiment

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def get_sentiment_vader(text):
    sid = SentimentIntensityAnalyzer()
    sent = sid.polarity_scores(text)
    compund = sent['compound']
    if compund > 1/3:
        return 'POSITIVE'
    elif compund < - 1/3:
        return 'NEGATIVE'
    else:
        return 'UNKOWN'

In [ ]:
sentiment_df_vader = text_series.apply(get_sentiment_vader)
sentiment_df_vader

0     NEGATIVE
1       UNKOWN
2       UNKOWN
3     NEGATIVE
4       UNKOWN
        ...   
95      UNKOWN
96    NEGATIVE
97    NEGATIVE
98      UNKOWN
99    NEGATIVE
Name: filename, Length: 100, dtype: object

In [ ]:
full_df_vader = pd.DataFrame([df['filename'][:100], text_series, sentiment_df_vader]).T
full_df_vader.columns=['filename', 'text', 'sentiment']
full_df_vader

,filename,text,sentiment
0,911_first6sec/call_2_0.wav,My mom had a bad time.,NEGATIVE
1,911_first6sec/call_8_0.wav,Hello. Hello. Is everything okay?,UNKOWN
2,911_first6sec/call_9_0.wav,I need a police officer over here at 7th. Wha...,UNKOWN
3,911_first6sec/call_10_0.wav,"Ma'am, my pickup was stolen. I had to go find...",NEGATIVE
4,911_first6sec/call_11_0.wav,No ma'am I don't. I don't have an emergency. ...,UNKOWN
...,...,...,...
95,911_first6sec/call_110_0.wav,"This is Gifford. Hello? Hello? Oh, number one...",UNKOWN
96,911_first6sec/call_111_0.wav,"across, forty seven oh nine fourty隔s avenue f...",NEGATIVE
97,911_first6sec/call_112_0.wav,"I have, um, I need somebody escorted out of m...",NEGATIVE
98,911_first6sec/call_114_0.wav,Hello? Hello? Hello? Hello 911? Hello?,UNKOWN


In [ ]:
sentiment_df_vader[sentiment_df_vader == 'NEGATIVE'].count()

21

In [ ]:
sentiment_df_vader[sentiment_df_vader == 'POSITIVE'].count()

20

In [ ]:
import torch

In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
%conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

^C

Note: you may need to restart the kernel to use updated packages.
